### 能返回异常就不要返回None

In [2]:
#看下面的例子:
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None

In [4]:
#这个函数可以这样用
x = 0
y = 5
result = divide(x, y)
if result is None:
    print('Invalid inputs')

In [5]:
#但是不能这样用！
x = 0
y = 5
result = divide(x, y)
#这是因为 0 会被判断为 False
if not result:
    print('Invalid inputs')

Invalid inputs


In [6]:
#最好的解决方案是,尽可能不要返回 None！
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs!') from e

### 闭包与变量作用域

In [2]:
#你想排序一个列表, 并且动态的指定一些特殊元素的优先级
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (0, x)
        return (1, x)
    values.sort(key = helper)

In [3]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6] 
group = {2, 3, 5, 7} 
sort_priority(numbers, group) 
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


In [9]:
#sort_priority 表明了Python语言的三个特点:
## 1. Python支持 闭包。函数可以引用在它定义范围内的变量
## 2. 函数是Python的一等公民, 你可以传递函数作为参数
## 3. Python按照字典序比较tuple类型的序

In [1]:
#在用函数嵌套的时候,你需要注意变量的作用域
def sort_priority2(numbers, group):
    found = False
    def helper(x):
        if x in group:
            #会在helper定义的范围内创建新的变量,
            #而不是赋值原来的变量
            found = True
            return (0, x)
        return (1, x)
    numbers.sort(key = helper)
    return found

In [4]:
found = sort_priority2(numbers, group)
print('Found:', found)
print(numbers)

Found: False
[2, 3, 5, 7, 1, 4, 6, 8]


In [6]:
#注意变量引用和赋值的区别！！！

#当引用一个变量时:
## ->. 首先查找当前函数的范围(如上例的helper)
## -->. 其次查找上层函数的范围(sort_priority2)
## --->. 接着查找全局作用域
## ---->. 最后查找 built-in 作用域
#赋值一个变量时:
## ->. 如果在当前的范围内存在,就直接赋值
## -->. 但如果不存在, Python会在当前作用域内创建一个新的变量！

In [7]:
#解决方案 1. nonlocal关键字
#但是最好不要用
def sort_priority3(numbers, group):
    found = False
    def helper(x):
        nonlocal found
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    numbers.sort(key = helper)
    return found 

In [8]:
#解决方案 2. Move your helper func to helper cls
class Sorter(object):
    def __init__(self, group):
        self.group = group
        self.found = False
    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)
sorter = Sorter(group)
numbers.sort(key = sorter)
assert sorter.found is True

### 如果返回的列表太大, 那么考虑返回生成器

In [10]:
#下面的函数统计了文本中每个单词的位置(索引)
def index_words(text):
    result = []
    if text:
        result.append(0)
    for index, letter in enumerate(text):
        if letter == ' ':
            result.append(index + 1)
    return result

In [11]:
address = 'Four score and seven years ago...'
result = index_words(address)
print(result[:3])

[0, 5, 11]


In [12]:
#上面代码有两个问题
##1. 代码过于dense和nosiy,核心代码只有一句
"""
result.append(index + 1)
"""
#解决第一个问题的方案: 用生成器
def index_words_iter(text):
    if text:
        yield 0
    for index, letter in enumerate(text):
        if letter == ' ':
            yield index + 1

In [13]:
#index_words的第二个问题
##2. result再返回之前都必须储存在list里, 对于大文本输入,这耗费内存
#解决方案:下面的函数每次读取一行然后生成这一行单词的位置,
#这样的好处是, 函数每次只需要读取一行,而不是一次性读取全部的文本。当然函数可以生成任意长度的文本的单词位置
def index_file(handle):
    #初始化index
    offset = 0
    #每次读取输入的每一行
    for line in handle:
        #生成行首单词的位置
        if line:
            yield offset
        #遍历之后的单词,遇到空格就返回当前单词的位置
        for letter in line:
            offset += 1
            if letter == ' ':
                yield offset

### 当你需要迭代参数时, 保持警惕,尤其当参数为生成器时

In [14]:
#下面的函数将列表中的数字标准化
def normalize(numbers):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

In [16]:
visits = [15, 35, 80]
percentages = normalize(visits)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [17]:
#下面假设numbers来自一个很大的文件, of course 我可以通过生成器:

# def read_visits(data_path):
#     with open(data_path) as f:
#         for line in f:
#             yield int(line)

In [18]:
#让我们模拟一个 生成器输入
numbers = (i for i in range(0, 1000))

In [19]:
#percentage返回一个空表
#这是因为当迭代器/生成器抛出 StopIteration 异常后, 它的状态就已经 over了。
#normalize函数中的 total = sum(numbers)使得迭代器终止。
percentages = normalize(numbers)
print(percentages)
#有简单的解决方案, 比如在normalize里copy the iterator,但这么做会消耗内存。

[]


In [23]:
#解决方案1.
## 在函数内部新建迭代器

#匿名函数numbers
numbers = lambda : (i for i in range(0, 10))

def normalize_func(get_iter):
    total = sum(get_iter())
    result = []
    for value in get_iter():
        percent = 100 * value / total
        result.append(percent)
    return result

In [24]:
percentages = normalize_func(numbers)
print(percentages)

[0.0, 2.2222222222222223, 4.444444444444445, 6.666666666666667, 8.88888888888889, 11.11111111111111, 13.333333333333334, 15.555555555555555, 17.77777777777778, 20.0]


In [ ]:
#上面的方法 works, 但是传递匿名函数显得冗余
#解决方案2.
## 构建自己的迭代器 class

In [ ]:
#关于迭代器 class
##需要提供 __iter__方法:
### for x in foo 等价于 foo,__iter__
### __iter__方法返回一个可迭代对象, 这个可迭代对象有自己的 __next__方法
class ReadVisits(object):
    def __init__(self, data_path):
        self.data_path = data_path
    def __iter__(self):
        with open(self.data_path) as f:
            for line in f:
                yield int(line)

In [ ]:
# PS: 你可以让你的normalize函数更加defensive:
def normalize_defensive(numbers):
    #如果是迭代器对象, 那么 __iter__方法将返回对象本身
    #如果是容器对象, iter方法将返回新的可迭代对象
    if iter(numbers) is iter(numbers):
        raise TypeError('Must supply a container!')
    pass

### 用 *args来打包函数的参数

In [25]:
def log(message, *values):
    if not values:
        print(message)
    else:
        values_str = ','.join([str(x) for x in values])
        print('%s : %s' % (message, values_str))

In [26]:
#log函数使用起来很灵活
log('My numbers are', 1, 2)
log('Hi there')
log('Favorite colors', *[1,2,3,4,5])

My numbers are : 1,2
Hi there
Favorite colors : 1,2,3,4,5


In [27]:
# *args的两个问题:
##1. *args 的变量参数在传递进函数之前会被打包成tuple。因此如果你传递了一个生成器, 他会迭代直到终止
##2. *args 使用之后你无法在为你函数增加新的位置参数。

#下面为log函数1增加新的位置参数 message
def log(sequence, message, *values):
    if not values:
        print('%s : %s' % (sequence, message))
    else:
        values_str = ','.join([str(x) for x in values])
        print('%s : %s : %s' % (sequence, message, values_str))

In [28]:
log(1, 'Favorite', 7, 33) #New usage is OK
log('Favorite', 7, 33)#But old usage is not OK

1 : Favorite : 7,33
Favorite : 7 : 33


### 为关键字参数增加默认值

In [30]:
#在函数调用时, 位置参数必须放在关键字参数前面。
#在函数调用时, 使用关键字参数增加代码的可读性
#你可以在函数定义时指定默认值,这会简化你的函数调用。如果你打算用默认值那么可以在调用时忽略他们,
#否则使用关键字参数。

### 用 None 来赋值"动态默认参数"

In [ ]:
#默认值只会执行一次,就是在函数定义的时候。因此对于动态的值比如{}和[],
#这一行为会引发麻烦

In [9]:
#来看下面的例子
from datetime import datetime
def log(message = 'haha', when = datetime.now()):
    print('%s : %s' % (when, message))


In [10]:
log()

2019-03-16 13:43:30.252374 : haha


In [11]:
from time import sleep
sleep(.5)
log()

2019-03-16 13:43:30.252374 : haha


In [12]:
log('lol')

2019-03-16 13:43:30.252374 : lol


In [13]:
#上面三次调用中, 时间戳是相通的.
#因为when的值, datetime.now 只在函数定义的时候执行一次！！！
#解决方案: 用 None
def log(message, when = None):
    when = datetime.now() if when is None else when
    print('%s : %s' % (when, message))

In [14]:
log('Hi there')
sleep(.5)
log('Hi again')

2019-03-16 14:34:32.428303 : Hi there
2019-03-16 14:34:32.931494 : Hi again


In [16]:
#None 还有一个用处,当参数值是不可变的,你得用None做默认值
import json
def decode(data, default = {}):
    try:
        return json.loads(data)
    except ValueError:
        return default

In [17]:
foo = decode('bad data')
foo['stuff'] = 5
bar = decode('also bad')
bar['meep'] = 1

In [18]:
print('Foo :', foo)
print('Bar :', bar)

Foo : {'stuff': 5, 'meep': 1}
Bar : {'stuff': 5, 'meep': 1}


In [19]:
#foo 和 bar 是同一个字典！！！！
assert foo is bar

In [20]:
#因此你得用 None 作为 default的默认值
def decode(data, default = None):
    try:
        return json.loads(data)
    except ValueError:
        return default

### 用强制关键字参数是你的代码更加清晰

In [21]:
# * 之后的全部是强制关键字参数
def safe_division(number, divisor, *,
                  ignore_overflow = False,
                  ignore_zero_division = False):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [22]:
#无法通过位置参数赋值
safe_division(1, 10, True, False)

TypeError: safe_division() takes 2 positional arguments but 4 were given

In [23]:
#但是可以使用默认值
safe_division(1, 10)

0.1